In [68]:
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import deformetrica as dfca
import vtk
import csv

In [42]:
def read_vtk_file(file_path):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    poly_data = reader.GetOutput()

    points_vtk = poly_data.GetPoints()
    points = [points_vtk.GetPoint(i) for i in range(points_vtk.GetNumberOfPoints())]

    lines_vtk = poly_data.GetLines()
    lines_vtk.InitTraversal()
    lines = []
    idList = vtk.vtkIdList()
    while lines_vtk.GetNextCell(idList):
        line = [idList.GetId(i) for i in range(idList.GetNumberOfIds())]
        lines.append(line)

    return points, lines

In [43]:

data_path = "testfile_24-01-27/"
#data_base = os.path.join(data_path, '/data/')
#vtk_files = glob.glob(data_base + "*.vtk")

aneurisk_001 = os.path.join(data_path, "alignedC0001.vtk")
aneurisk_002 = os.path.join(data_path, "alignedC0002.vtk")
aneurisk_001_51 = os.path.join(data_path, "alignedC0001_51.vtk")
temp_shapes = os.path.join(data_path, "BH0017_Right_new_new.vtk")

points, id = read_vtk_file(aneurisk_001)

# 现在，points和lines数组包含了点的坐标和连线信息


In [44]:
id

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63]]

In [56]:
def write_vtk_file(file_path, points, lines):
    poly_data = vtk.vtkPolyData()

    points_vtk = vtk.vtkPoints()
    for point in points:
        points_vtk.InsertNextPoint(point)
    poly_data.SetPoints(points_vtk)

    lines_vtk = vtk.vtkCellArray()
    for line in lines:
        line_id_list = vtk.vtkIdList()
        for pid in line:
            line_id_list.InsertNextId(pid)
        lines_vtk.InsertNextCell(line_id_list)
    poly_data.SetLines(lines_vtk)

    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(file_path)
    writer.SetInputData(poly_data)
    writer.Write()


def clip_vtk(file_path, cut_id, output_file):
    points, lines = read_vtk_file(file_path)

    # 保留 cut_id 之前的点
    clipped_points = points[:cut_id + 1]

    # 创建一条新的线段，包含从起始点到 cut_id 的所有点
    clipped_lines = [[i for i in range(cut_id + 1)]]

    # 写入新的 VTK 文件
    write_vtk_file(output_file, clipped_points, clipped_lines)

    # 打印 cut_id 和对应的点坐标
    cut_point = clipped_points[-1] if clipped_points else None
    print("Cut ID:", cut_id)
    print("Cut point coordinates:", cut_point)

    # 打印剪切后的总点数
    print("Number of points after clipping:", len(clipped_points))

    #return cut_point



# 剪切曲线
cut_id = 52  # 设置你的 cut_id
input_vtk_file = aneurisk_001_51
output_vtk_file = '1111_51.vtk'
clip_vtk(input_vtk_file, cut_id, output_vtk_file)


Cut ID: 52
Cut point coordinates: (2.6750800609588623, -10.993000030517578, 11.406800270080566)
Number of lines after clipping: 1
Number of points after clipping: 53


In [ ]:
# # vtk 按文件名创建 CSV 文件
# def create_csv_from_vtk_folder(folder_path, csv_file_path):
#     # 获取文件夹中所有的 vtk 文件，并按名称排序
#     vtk_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.vtk')])

#     # 定义 CSV 文件的列标题
#     headers = ['Name', 'cut_point']

#     # 创建并写入 CSV 文件
#     with open(csv_file_path, 'w', newline='', encoding='ascii') as file:
#         writer = csv.writer(file)
#         writer.writerow(headers)

#         # 对于每个 vtk 文件，写入文件名和空的列
#         for vtk_file in vtk_files:
#             file_name_without_extension = os.path.splitext(vtk_file)[0]
#             row = [file_name_without_extension] + [''] * (len(headers) - 1)
#             writer.writerow(row)

# # 指定文件夹路径和 CSV 文件的路径
# folder_path = 'aneurisk/'  # 替换为您的 VTK 文件夹路径
# csv_file_path = 'aneurisk_cut_point.csv'  # 您希望创建的 CSV 文件路径

# create_csv_from_vtk_folder(folder_path, csv_file_path)

In [69]:

# 读取 CSV 文件
def read_cut_points(csv_file):
    cut_points = {}
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # 跳过标题行
        for row in reader:
            if row[1]:  # 确保 cut_id 不是空字符串
                filename = row[0] + '.vtk'
                cut_id = int(row[1])
                cut_points[filename] = cut_id
            else:
                print(f"Warning: No cut_id for file {row[0]}")
    return cut_points
cut_points = read_cut_points('aneurisk_cut_point_24-01-17.csv')


In [71]:
input_folder = 'aneurisk'
output_folder = 'aneurisk_clipped'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 对每个文件进行剪切处理
for file, cut_id in cut_points.items():
    input_file = os.path.join(input_folder, file)
    output_file = os.path.join(output_folder, file)
    clip_vtk(input_file, cut_id, output_file)
    print(f"Processed {file} with cut_id {cut_id}")

Cut ID: 57
Cut point coordinates: (5.749372482299805, -14.201430320739746, 9.061824798583984)
Number of lines after clipping: 1
Number of points after clipping: 58
Processed alignedC0001.vtk with cut_id 57
Cut ID: 59
Cut point coordinates: (9.071993827819824, -11.517430305480957, 7.557266712188721)
Number of lines after clipping: 1
Number of points after clipping: 60
Processed alignedC0002.vtk with cut_id 59
Cut ID: 57
Cut point coordinates: (7.994793891906738, -11.798277854919434, 8.288204193115234)
Number of lines after clipping: 1
Number of points after clipping: 58
Processed alignedC0004.vtk with cut_id 57
Cut ID: 58
Cut point coordinates: (5.25706148147583, -12.489356994628906, 10.722930908203125)
Number of lines after clipping: 1
Number of points after clipping: 59
Processed alignedC0005.vtk with cut_id 58
Cut ID: 57
Cut point coordinates: (6.941620826721191, -6.056509017944336, 8.818439483642578)
Number of lines after clipping: 1
Number of points after clipping: 58
Processed ali

In [73]:
# # 对剪切后的曲线差值（样条差值，不是均等的）
# def resample_polyline(poly_data, num_points):
#     resample = vtk.vtkPolyData()
#     points = poly_data.GetPoints()
#     resampled_points = vtk.vtkPoints()
    
#     # 创建三个单独的样条插值器：一个用于X坐标，一个用于Y坐标，一个用于Z坐标
#     spline_x = vtk.vtkCardinalSpline()
#     spline_y = vtk.vtkCardinalSpline()
#     spline_z = vtk.vtkCardinalSpline()

#     # 为每个坐标轴添加点
#     for i in range(points.GetNumberOfPoints()):
#         x, y, z = points.GetPoint(i)
#         spline_x.AddPoint(i, x)
#         spline_y.AddPoint(i, y)
#         spline_z.AddPoint(i, z)

#     # 使用样条插值器计算重新采样后的点坐标
#     for i in range(num_points):
#         t = i * (points.GetNumberOfPoints() - 1) / (num_points - 1)
#         x = spline_x.Evaluate(t)
#         y = spline_y.Evaluate(t)
#         z = spline_z.Evaluate(t)
#         resampled_points.InsertNextPoint(x, y, z)

#     resample.SetPoints(resampled_points)

#     # 创建线段，连接重新采样后的点
#     lines = vtk.vtkCellArray()
#     line = vtk.vtkPolyLine()
#     line.GetPointIds().SetNumberOfIds(num_points)
#     for i in range(num_points):
#         line.GetPointIds().SetId(i, i)
#     lines.InsertNextCell(line)
#     resample.SetLines(lines)

#     return resample


# # 缩放曲线
# def scale_polyline(poly_data, target_distance):
#     points = poly_data.GetPoints()
#     scaled_points = vtk.vtkPoints()
#     for i in range(points.GetNumberOfPoints() - 1):
#         p1 = np.array(points.GetPoint(i))
#         p2 = np.array(points.GetPoint(i + 1))
#         direction = p2 - p1
#         normalized_direction = direction / np.linalg.norm(direction)
#         new_point = p1 + target_distance * normalized_direction
#         scaled_points.InsertNextPoint(*new_point)
#     scaled_points.InsertNextPoint(*points.GetPoint(points.GetNumberOfPoints() - 1))  # Add last point
#     poly_data.SetPoints(scaled_points)
#     return poly_data

# # 处理所有文件
# def process_files(input_folder, output_folder, num_points=64, point_distance=1.0):
#     for file in os.listdir(input_folder):
#         if file.endswith('.vtk'):
#             reader = vtk.vtkPolyDataReader()
#             reader.SetFileName(os.path.join(input_folder, file))
#             reader.Update()
#             poly_data = reader.GetOutput()

#             # 重新采样
#             resampled = resample_polyline(poly_data, num_points)

#             # 缩放
#             scaled = scale_polyline(resampled, point_distance)

#             # 写入文件
#             writer = vtk.vtkPolyDataWriter()
#             writer.SetFileName(os.path.join(output_folder, file))
#             writer.SetInputData(scaled)
#             writer.Write()

# # 执行处理
# input_folder = 'aneurisk_clipped'
# output_folder = 'aneurisk_clipped_resampled'

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# process_files(input_folder, output_folder)


In [83]:

def compute_polyline_length(polyline):
    total_length = 0.0
    points = polyline.GetPoints()
    for i in range(1, points.GetNumberOfPoints()):
        p1 = np.array(points.GetPoint(i - 1))
        p2 = np.array(points.GetPoint(i))
        total_length += np.linalg.norm(p2 - p1)
    return total_length


def scale_polyline(polyline, target_length):
    points = polyline.GetPoints()
    original_length = compute_polyline_length(points)
    scale_factor = target_length / original_length

    scaled_points = vtk.vtkPoints()
    for i in range(points.GetNumberOfPoints()):
        p = np.array(points.GetPoint(i))
        scaled_p = p * scale_factor
        scaled_points.InsertNextPoint(scaled_p.tolist())

    scaled_polyline = vtk.vtkPolyData()
    scaled_polyline.SetPoints(scaled_points)

    # 保留原始线段连接
    lines = vtk.vtkCellArray()
    line = vtk.vtkPolyLine()
    line.GetPointIds().SetNumberOfIds(points.GetNumberOfPoints())
    for i in range(points.GetNumberOfPoints()):
        line.GetPointIds().SetId(i, i)
    lines.InsertNextCell(line)
    scaled_polyline.SetLines(lines)

    return scaled_polyline


def resample_polyline(polyline, num_points):
    length = compute_polyline_length(polyline)
    points = polyline.GetPoints()
    resampled_points = vtk.vtkPoints()
    resampled_polyline = vtk.vtkPolyLine()
    resampled_polyline.GetPointIds().SetNumberOfIds(num_points)

    resampled_points.InsertNextPoint(points.GetPoint(0))
    accum_length = 0.0
    next_point_index = 1
    for i in range(1, num_points - 1):
        target_length = i * length / (num_points - 1)
        while accum_length < target_length and next_point_index < points.GetNumberOfPoints():
            p1 = np.array(points.GetPoint(next_point_index - 1))
            p2 = np.array(points.GetPoint(next_point_index))
            segment_length = np.linalg.norm(p2 - p1)
            if accum_length + segment_length > target_length:
                t = (target_length - accum_length) / segment_length
                new_point = p1 + t * (p2 - p1)
                resampled_points.InsertNextPoint(new_point)
                break
            accum_length += segment_length
            next_point_index += 1
    resampled_points.InsertNextPoint(points.GetPoint(points.GetNumberOfPoints() - 1))

    for i in range(num_points):
        resampled_polyline.GetPointIds().SetId(i, i)

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(resampled_points)

    lines = vtk.vtkCellArray()
    lines.InsertNextCell(resampled_polyline)
    polydata.SetLines(lines)

    return polydata

# def process_files(input_folder, output_folder, num_points=64):
#     for file in os.listdir(input_folder):
#         if file.endswith('.vtk'):
#             reader = vtk.vtkPolyDataReader()
#             reader.SetFileName(os.path.join(input_folder, file))
#             reader.Update()
#             poly_data = reader.GetOutput()

#             # 重新采样
#             resampled = resample_polyline(poly_data, num_points)

#             # 写入文件
#             writer = vtk.vtkPolyDataWriter()
#             writer.SetFileName(os.path.join(output_folder, file))
#             writer.SetInputData(resampled)
#             writer.Write()

def process_files(input_folder, output_folder, num_points=64, point_distance=1.0):
    target_length = (num_points - 1) * point_distance
    for file in os.listdir(input_folder):
        if file.endswith('.vtk'):
            reader = vtk.vtkPolyDataReader()
            reader.SetFileName(os.path.join(input_folder, file))
            reader.Update()
            poly_data = reader.GetOutput()

            # 缩放
            scaled = scale_polyline(poly_data, target_length)

            # 重新采样
            resampled = resample_polyline(scaled, num_points)

            # 写入文件
            writer = vtk.vtkPolyDataWriter()
            writer.SetFileName(os.path.join(output_folder, file))
            writer.SetInputData(resampled)
            writer.Write()


# 执行处理
input_folder = 'aneurisk_clipped'
output_folder = 'aneurisk_resampled11'

# 如果输出文件夹不存在，则创建它
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

process_files(input_folder, output_folder)


TypeError: GetPoints() takes exactly 2 arguments (0 given)

In [86]:
import vtk
import numpy as np
import os

# 计算 vtkPoints 对象中的点所构成的多段线的总长度
def compute_polyline_length(points):
    total_length = 0.0
    for i in range(1, points.GetNumberOfPoints()):
        p1 = np.array(points.GetPoint(i - 1))
        p2 = np.array(points.GetPoint(i))
        total_length += np.linalg.norm(p2 - p1)
    return total_length

# 缩放多段线，使其总长度达到 target_length
def scale_polyline(polyline, target_length):
    points = polyline.GetPoints()
    original_length = compute_polyline_length(points)
    scale_factor = target_length / original_length

    scaled_points = vtk.vtkPoints()
    for i in range(points.GetNumberOfPoints()):
        p = np.array(points.GetPoint(i))
        scaled_p = p * scale_factor
        scaled_points.InsertNextPoint(scaled_p.tolist())

    scaled_polyline = vtk.vtkPolyData()
    scaled_polyline.SetPoints(scaled_points)

    # 保留原始线段连接
    lines = vtk.vtkCellArray()
    line = vtk.vtkPolyLine()
    line.GetPointIds().SetNumberOfIds(points.GetNumberOfPoints())
    for i in range(points.GetNumberOfPoints()):
        line.GetPointIds().SetId(i, i)
    lines.InsertNextCell(line)
    scaled_polyline.SetLines(lines)

    return scaled_polyline

# 将 vtkPolyData 对象中的多段线重新采样为具有 num_points 个点的新多段线
def resample_polyline(polyline, num_points):
    points = polyline.GetPoints()
    length = compute_polyline_length(points)
    resampled_points = vtk.vtkPoints()
    resampled_polyline = vtk.vtkPolyLine()
    resampled_polyline.GetPointIds().SetNumberOfIds(num_points)

    resampled_points.InsertNextPoint(points.GetPoint(0))
    accum_length = 0.0
    next_point_index = 1
    for i in range(1, num_points - 1):
        target_length = i * length / (num_points - 1)
        while accum_length < target_length and next_point_index < points.GetNumberOfPoints():
            p1 = np.array(points.GetPoint(next_point_index - 1))
            p2 = np.array(points.GetPoint(next_point_index))
            segment_length = np.linalg.norm(p2 - p1)
            if accum_length + segment_length > target_length:
                t = (target_length - accum_length) / segment_length
                new_point = p1 + t * (p2 - p1)
                resampled_points.InsertNextPoint(new_point)
                break
            accum_length += segment_length
            next_point_index += 1
    resampled_points.InsertNextPoint(points.GetPoint(points.GetNumberOfPoints() - 1))

    for i in range(num_points):
        resampled_polyline.GetPointIds().SetId(i, i)

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(resampled_points)

    lines = vtk.vtkCellArray()
    lines.InsertNextCell(resampled_polyline)
    polydata.SetLines(lines)

    return polydata

# 处理指定文件夹中的所有 vtk 文件，先缩放然后重新采样
def process_files(input_folder, output_folder, num_points=64, point_distance=1.0):
    target_length = (num_points) * point_distance
    for file in os.listdir(input_folder):
        if file.endswith('.vtk'):
            reader = vtk.vtkPolyDataReader()
            reader.SetFileName(os.path.join(input_folder, file))
            reader.Update()
            poly_data = reader.GetOutput()

            # 缩放
            scaled = scale_polyline(poly_data, target_length)

            # 重新采样
            resampled = resample_polyline(scaled, num_points)

            # 写入文件
            writer = vtk.vtkPolyDataWriter()
            writer.SetFileName(os.path.join(output_folder, file))
            writer.SetInputData(resampled)
            writer.Write()

# 执行处理
input_folder = 'aneurisk_clipped'  # 输入文件夹
output_folder = 'aneurisk_resampled111'  # 输出文件夹

# 如果输出文件夹不存在，则创建它
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

process_files(input_folder, output_folder)


In [89]:

def read_vtk_file_to_polydata(file_path):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    return reader.GetOutput()

# 测试 VTK 文件长度的函数
def test_vtk_file_length(file_path):
    polydata = read_vtk_file_to_polydata(file_path)
    length = compute_polyline_length(polydata)
    return length

# 使用示例
vtk_file = "aneurisk_clipped/alignedC0002.vtk"
length = test_vtk_file_length(vtk_file)
print("Length of the polyline:", length)
p, d = read_vtk_file(vtk_file)
print("Number of points:", len(p))
print("Number of lines:", d)

vtk_file = "aneurisk_resampled/alignedC0002.vtk"
length = test_vtk_file_length(vtk_file)
print("Length of the polyline:", length)
p, d = read_vtk_file(vtk_file)
print("Number of points:", len(p))
print("Number of lines:", d)

vtk_file = "aneurisk_resampled111/alignedC0006.vtk"
length = test_vtk_file_length(vtk_file)
print("Length of the polyline:", length)
p, d = read_vtk_file(vtk_file)
print("Number of points:", len(p))
print("Number of lines:", d)

vtk_file = "vmtk64a/BH0004_Left_new_new.vtk"
length = test_vtk_file_length(vtk_file)
print("Length of the polyline:", length)
p, d = read_vtk_file(vtk_file)
print("Number of points:", len(p))
print("Number of lines:", d)

Length of the polyline: 58.856081240285796
Number of points: 60
Number of lines: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]]
Length of the polyline: 58.37099557233955
Number of points: 64
Number of lines: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]]
Length of the polyline: 63.33221034086063
Number of points: 64
Number of lines: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]]
Length of the polyline: 63.9

In [98]:
import vtk
import numpy as np

def read_vtk_file(file_path):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    return reader.GetOutput()

def calculate_distance_between_points(poly_data, point_id1, point_id2):
    points = poly_data.GetPoints()
    p1 = np.array(points.GetPoint(point_id1))
    p2 = np.array(points.GetPoint(point_id2))
    distance = np.linalg.norm(p2 - p1)
    print(p2)
    print(p1)
    return distance

# 使用示例
vtk_file_path = 'vmtk64a/BH0017_Right_new_new.vtk'  # 替换为你的 VTK 文件路径
point_id1 = 0  # 第一个点的索引
point_id2 = 1  # 第二个点的索引

poly_data = read_vtk_file(vtk_file_path)
distance = calculate_distance_between_points(poly_data, point_id1, point_id2)
print(f"Distance between points {point_id1} and {point_id2}: {distance}")


[-1.01696002  0.115449    0.129632  ]
[0. 0. 0.]
Distance between points 0 and 1: 1.0316688506067517


In [95]:
abs(-1.01696*-1.01696 + 0.115449*0.115449 + 0.129632* 0.129632)

1.064340568625